# Ticket aggregator

## Step 1: base classificator

In [1]:
# Constants

PRICE_MESSAGE = '- What is the price of ticket in dollars?\n'
PRICE_CONFIRMATION_MESSAGE = lambda value: f'- Is this correct price: ${str(value)}?'
N_OF_TRNSF_MESSAGE = '- What is the number of transfers?\n'
N_OF_TRNSF_CONFIRMATION_MESSAGE = lambda value: f'- Is this correct number of transfers: {str(value)}?'
INCORRECT_SYMBOL_MESSAGE = '- You entered incorrect symbol.'
INCORRECT_VALUE_MESSAGE = "- It seems you didn't enter a value or enter incorrect one. Try again, please.\n"
REFUND_MESSAGE = '- Is there opportunity to refund the ticket?'
LUGGAGE_MESSAGE = '- Is luggage included in the ticket cost?'
TRNSF_DURATION_MESSAGE = '- What is duration of transfer?\nPlease, enter number of hours rounding to upper.\n'
TRNSF_DURATION_CONFIRMATION_MESSAGE = lambda value: f'- Is this correct duration of transfer: {str(value)}h?'
LUGGAGE_DIM_MESSAGE = """- What is acceptable dimensions of hand luggage?
Please, enter three numbers, divided with space.\n"""
LUGGAGE_DIM_CONFIRMATION_MESSAGE = lambda value: f'- Is this correct dimensions?: \
{str(value[0])} {str(value[1])} {str(value[2])}'
TRANSFER_TIME_MESSAGE = '- Is the transfer in daytime?'
BOOLEAN_QUESTION = lambda message: message + ' [Y/n] \n'

# Helpers
isExit = lambda value: value == 'e' or value == 'E'
isYes = lambda value: value == 'y' or value == 'Y' or value == ''
isNo = lambda value: value == 'n' or value == 'N'
bool_answer_to_str = lambda bool_answer: 'Yes' if bool_answer else 'No'
get_num_from_str = lambda string: ''.join(re.findall('\d+', string))

In [2]:
import re
import random
from typing import Type, TypeVar, Dict, Callable, List


def get_float(message: str) -> float or str:
    answer = input(message)
    # If user wants to exit, just pass this information onward
    if isExit(answer):
        return answer
    num: List[str] = answer.split('.')
    # We always have at least one element. It can be empty
    parsed_num = get_num_from_str(num[0])
    # But if we have more than one element, so there is a float point. Let's use it to create float number.
    # It can be more than one dot in the string, but it doesn't make a sence for numbers, so let's just ignore them
    if len(num) > 1:
        parsed_num = parsed_num + '.' + get_num_from_str(num[1])
    # if we don't have any digits in the input string, parsedNum is empty string, so let's return it as it
    # else - parsedNum is a valid number in string format, so let's convert it to float
    
    return parsed_num if parsed_num == '' else abs(float(parsed_num))

def get_integer(message: str) -> int or str:
    num: float or str = get_float(message)
    if isExit(num):
        return num
    # It is important to check exactly if num empty string, because num can be zero but zero should be handled as int
    return num if num == '' else abs(int(num))

def get_bool(message: str) -> bool or None or str:
    answer: str = input(BOOLEAN_QUESTION(message))
    # If user wants to exit, just pass this information onward
    if isExit(answer):
        return answer
    # If answer is "yes" or "no" return boolean
    elif isYes(answer) or isNo(answer):
        return isYes(answer) == True
    else:
    # If user enter wrong value return None and handle it later
        return None

In [3]:
# Create decorators and decorate common getters to pass custom messages

def get_price_decorator(get_func: Type[get_float]) -> Callable[[str], float or str]:
    def get_price_func(message: str = PRICE_MESSAGE) -> float or str:
        return get_func(message)
    return get_price_func


def get_price_confirmation_decorator(get_func: Type[get_bool]) -> Callable[[float, str], bool or None or str]:
    def get_price_confirmation_func(value: float, message: str) -> bool or None or str:
        if message == '':
            message = PRICE_CONFIRMATION_MESSAGE(value)
        return get_func(message)
    return get_price_confirmation_func


get_price = get_price_decorator(get_float)
get_price_confirmation = get_price_confirmation_decorator(get_bool)

def get_n_transfers_decorator(get_func: Type[get_integer]) -> Callable[[str], int or str]:
    def get_n_transfers_func(message: str = N_OF_TRNSF_MESSAGE) -> int or str:
        return get_func(message)
    return get_n_transfers_func


def get_n_transfers_confirmation_decorator(get_func: Type[get_bool]) -> Callable[[int, str], bool or None or str]:
    def get_n_transfers_confirmation_func(value: int, message: str) -> bool or None or str:
        if message == '':
            message = N_OF_TRNSF_CONFIRMATION_MESSAGE(value)
        return get_func(message)
    return get_n_transfers_confirmation_func


get_n_transfers = get_n_transfers_decorator(get_integer)
get_n_transfers_confirmation = get_n_transfers_confirmation_decorator(get_bool)

In [4]:
def get_boolean_criterion(message: str) -> bool or str:
    answer = get_bool(message)
    # If user wants to exit, just pass this information onward
    if isExit(answer):
        return answer
    # If answer in right format, return it
    elif answer == True or answer == False:
        return answer
    # If user enter wrong value, call yourself until user enter correct value
    else:
        return get_boolean_criterion(INCORRECT_SYMBOL_MESSAGE)

In [5]:
def get_numeric_criterion(value_getter: Type[get_float], confirmation_getter: Type[get_bool]) -> float or int or str:
    def get_value() -> float or int:
        value = value_getter()
        if isExit(value):
            return value
        # Request value until user enter correct value
        while value == '':
            value = value_getter(INCORRECT_VALUE_MESSAGE)
        return value

    def get_confirmation(value: float, message: str = '') -> float or int or str:
        # Since we clear the value and compose the final result at its discretion, let's clarify
        # if we understand user right way
        answer = confirmation_getter(value, message)
        if isExit(answer):
            return answer
        if answer == True:
            return value
        elif answer == False:
            value = get_value()
            return get_confirmation(value)
        else:
            return get_confirmation(value, INCORRECT_SYMBOL_MESSAGE)
        
    value = get_value()
    if isExit(value):
        return value
    return get_confirmation(value)

In [6]:
def analizer(price: float, n_transfers: int, refund: bool, luggage: bool) -> str:
    if price < 200 and n_transfers < 2 and refund and luggage:
        return 'The best'
    elif price >= 200 and price <= 250 and n_transfers < 3:
        return 'Good enough'
    elif price > 250 and n_transfers > 2:
        return 'Bad'
    else:
        return 'Other'

In [46]:
def interface():        
    # Since we forced to check after each criteria does user want to exit, it
    # is more convenient to request information in loop, so let's create the array for it
    getters_array = [
        {
            'name': 'price',
            'get_value': get_numeric_criterion,
            'get_value_args': [get_price, get_price_confirmation]
        },
        {
            'name': 'n_transfers',
            'get_value': get_numeric_criterion,
            'get_value_args': [get_n_transfers, get_n_transfers_confirmation]
        },
        {
            'name': 'refund',
            'get_value': get_boolean_criterion,
            'get_value_args': [REFUND_MESSAGE]
        },
        {
            'name': 'luggage',
            'get_value': get_boolean_criterion,
            'get_value_args': [LUGGAGE_MESSAGE]
        },
    ]
    
    values = {}
    
    for item in getters_array:
        value = item['get_value'](*item['get_value_args'])
        if isExit(value):
            return
        values[item['name']] = value
    
    category = analizer(**values)
    
    
    print(
        '\n-------------------------------------------\n' +
        'OFFER FEATURES' +
        '\n-------------------------------------------\n' +
        '{:34s}  ${:2.2f}\n'.format('Price:', values['price']) +
        '{:34s}  {:1d}\n'.format('Number of transfers:', values['n_transfers']) +
        '{:34s}  {:2s}\n'.format('Opportunity to refund:', bool_answer_to_str(values['refund'])) +
        '{:34s}  {:2s}\n'.format('Luggage is included:', bool_answer_to_str(values['luggage'])) +
        '\n-------------------------------------------\n' +
        '{:34s}  {:7s}\n'.format('Offer category:', category)
    )

To start analyzer run next cell. You should answer several questions. To exit press "e" on any step.

In [47]:
interface()

- What is the price of ticket in dollars?
567
- Is this correct price: $567.0? [Y/n] 

- What is the number of transfers?
6
- Is this correct number of transfers: 6? [Y/n] 

- Is there opportunity to refund the ticket? [Y/n] 

- Is luggage included in the ticket cost? [Y/n] 


-------------------------------------------
OFFER FEATURES
-------------------------------------------
Price:                              $567.00
Number of transfers:                6
Opportunity to refund:              Yes
Luggage is included:                Yes

-------------------------------------------
Offer category:                     Bad    



## Step 2: analyzing

Our aggregator use only four criteria in very the simple way, but I think these are the most important criteria for users. On my observations the main part of aggregators use the same features, but with more complicated logic. So advantages for this aggregator are:
    - it uses popular criteria that are relevant for users
    - it implements simple logic so do not need many time and memory for execute and simply can be used in runtime
    - it does not require many data, it can classify even if we have information about only one ticket
    
On the other side obviously our aggregator is too simple and cannot react on the changes of input data. For example, if tickets will go up and be outside the maximum threshold, classification will give only "Bad" or "Other" categories even if it is the cheapiest ticket of all. So, I see next disadvantages:
    - the conditions are hard coded and do not adapt for data changes
    - we use limited number of criteria and do not pay attention to some other that can be usefull too
    - it cannot evaluate and take into consideration real rules and users preferences

## Step 3: alternative solution

For me price is important like for many other users and of course luggage too, but I do not like travel with big bag and hand over it in the luggage compartment. It steals many time for waiting my stuff. Usually I want to take with me my backpack inside the plane, so acceptable dimensions of hand luggage is critical for me.

It is cool to get to the destination by one flight, but one transfer is not a big problem. However if time between the landing and the next departure is too short you risk to late on your next flight. So I prefer to have not less than one and half hour between flights. On the other hand, stuck in the airport for 4 or 5 hours is not a nice perspective, but if you have 8 or more hours to your next flight, you can use this time to look out the city. So let's think that less than two hours and more than three, but less than eight hours between the flights is bad, else - good offer.

Let's code it

In [8]:
# Decorate base getters to get transfer duration

def get_transfer_duration_dec(get_func: Type[get_integer]) -> Callable[[str], int or str]:
    def get_transfer_duration_func(message: str = TRNSF_DURATION_MESSAGE) -> int or str:
        return get_func(message)
    return get_transfer_duration_func


def get_transfer_duration_confirmation_dec(get_func: Type[get_bool]) -> Callable[[int, str], bool or None or str]:
    def get_transfer_duration_confirmation_func(value: int, message: str) -> bool or None or str:
        if message == '':
            message = TRNSF_DURATION_CONFIRMATION_MESSAGE(value)
        return get_func(message)
    return get_transfer_duration_confirmation_func

get_transfer_duration = get_transfer_duration_dec(get_integer)
get_transfer_duration_confirmation = get_transfer_duration_confirmation_dec(get_bool)

In [9]:
def get_luggage_dimensions(message: str = LUGGAGE_DIM_MESSAGE) -> List[int] or str:
    answer: str = input(message)
    if isExit(answer):
        return answer
    
    dimensions: List[str] = list(map(get_num_from_str, answer.split(' ')))[:3]
        
    if len(dimensions) == 3 and all(dimensions):
        return list(map(int, dimensions))
    else:
        return ''
        
def get_luggage_dimensions_confirmation_dec(get_func: Type[get_bool]) -> bool or None or str:
    def get_luggage_dimensions_confirmation_func(value: List[int], message: str) -> bool or None or str:
        if message == '':
            message = LUGGAGE_DIM_CONFIRMATION_MESSAGE(value)
        return get_func(message)
    return get_luggage_dimensions_confirmation_func

get_luggage_dimensions_confirmation = get_luggage_dimensions_confirmation_dec(get_bool)

In [60]:
def new_analizer(
    price: float,
    n_transfers: int,
    is_transfer_daytime: bool,
    transfer_duration: int,
    luggage_dimentions: List[int],
):
    short_duration = transfer_duration and transfer_duration >= 2 and transfer_duration <= 3
    long_duration = transfer_duration and transfer_duration > 8
    the_best_transfer = n_transfers == 0 or (n_transfers == 1 and is_transfer_daytime and long_duration)
    good_anough_transfer = n_transfers == 1 and is_transfer_daytime and short_duration
    acceptable_luggage_dim = sum(luggage_dimentions) >= 50+40+23
    
    if price < 200 and the_best_transfer and acceptable_luggage_dim:
        return 'The Best'
    elif price < 250 and (the_best_transfer or good_anough_transfer) and acceptable_luggage_dim:
        return 'Good enough'
    else:
        return 'Bad'

In [61]:
def new_interface():        
    # Now we will to request criteria accordingly with special logic, so
    # will be more convenient to use key-value storage instead of array
    getters_obj = {
        'price': {
            'get_value': get_numeric_criterion,
            'get_value_args': [get_price, get_price_confirmation]
        },
        'n_transfers': {
            'get_value': get_numeric_criterion,
            'get_value_args': [get_n_transfers, get_n_transfers_confirmation]
        },
        'is_transfer_daytime': {
            'get_value': get_boolean_criterion,
            'get_value_args': [TRANSFER_TIME_MESSAGE]
        },
        'transfer_duration': {
            'get_value': get_numeric_criterion,
            'get_value_args': [get_transfer_duration, get_transfer_duration_confirmation]
        },
        'luggage_dimentions': {
            'get_value': get_numeric_criterion,
            'get_value_args': [get_luggage_dimensions, get_luggage_dimensions_confirmation]
        },
    }
    
    values = {}
    
    for key in getters_obj:
        if (key == 'is_transfer_daytime' and values['n_transfers'] != 1
        ) or (
            key == 'transfer_duration' and not values['is_transfer_daytime']
        ):
            values[key] = None
            continue
        item = getters_obj[key]
        value = item['get_value'](*item['get_value_args'])
        if isExit(value):
            return value
        values[key] = value
    
    category = new_analizer(**values)
    
    print(
        '\n-----------------------------------------------------\n' +
        'OFFER FEATURES' +
        '\n-----------------------------------------------------\n' +
        '{:40s}  ${:2.2f}\n\n'.format('Price:', values['price']) +
        '{:40s}  {:1d}\n'.format('Number of transfers:', values['n_transfers'])
    )
    
    if values['is_transfer_daytime']:
        print('{:40s}  {:2s}\n'.format(
            'Is the transfer in daytime?:', bool_answer_to_str(values['is_transfer_daytime'])
        ))
        
    if values['transfer_duration']:
        print('{:40s}  {:1d}h\n'.format('Transfer duration:', values['transfer_duration']))
        
    print(
        '{:40s}  {:1d}x{:1d}x{:1d}\n'.format(
            'Acceptable luggage dimentions:',
            values['luggage_dimentions'][0],
            values['luggage_dimentions'][1],
            values['luggage_dimentions'][2]
        ) + '\n-----------------------------------------------------\n' +
        '{:40s}  {:7s}\n'.format('Offer category:', category)
    )

To start analyzer run next cell. You should answer several questions. To exit press "e" on any step.

In [62]:
new_interface()

- What is the price of ticket in dollars?
233
- Is this correct price: $233.0? [Y/n] 

- What is the number of transfers?
1
- Is this correct number of transfers: 1? [Y/n] 

- Is the transfer in daytime? [Y/n] 

- What is duration of transfer?
Please, enter number of hours rounding to upper.
2
- Is this correct duration of transfer: 2h? [Y/n] 

- What is acceptable dimensions of hand luggage?
Please, enter three numbers, divided with space.
40 60 25
- Is this correct dimensions?: 40 60 25 [Y/n] 


-----------------------------------------------------
OFFER FEATURES
-----------------------------------------------------
Price:                                    $233.00

Number of transfers:                      1

Is the transfer in daytime?:              Yes

Transfer duration:                        2h

Acceptable luggage dimentions:            40x60x25

-----------------------------------------------------
Offer category:                           Good enough

